# The reduced decision making model by Wong-Wang
The equations and parameters are described in the body, appendix and supplementary materials of the following paper:

<i>Wong, Kong-Fatt and Xiao-Jing Wang.</i> <b>"A recurrent network mechanism  of time integration in perceptual decisions"</b>. Journal of Neuroscience 26, no. 4 (2006)

In [ ]:
import random

#Parameters
mu_0 = 0 #input strength; motion strength
c = 0 #input coherence

#Parameters to the input-output function
a = 270 #[(VnC)^{-1}]
b = 108 #[Hz]
d = 0.154 #[s]

#Kinetic parameters
g = 0.641 #dimensionless, called gamma in the paper
tau_S = 100 #[ms]
tau_AMPA = 2 #[ms]

#Synaptic coupling (population autapses {11,22} and between populations {21,12})
#JN11 = 0.2609
#JN22 = 0.2609
self_coupling = 0.2609
#JN21 = 0.0497
#JN12 = 0.0497
outer_coupling = 0.0497
couplings = [self_coupling,outer_coupling]

#Inputs from other sources (inhibitory and neutral populations: JA_ext; external: I0)
JA_ext = 5.2e-4
I0 = 0.3255

#Simulation noise
s_noise=0

#Single-cell input-output relation [Appendix & Supplementary material D]
def input_output_relation(state,a=270,b=108,d=0.154):
    return (a*state - b)/(1-np.exp(-d*(a*state - b)))
#H(x) = (a * x - b) / (1 - exp(-d*(a * x - b)))

#Visual motion stimulus to to the i-th {1,2} population 
I1 = JA_ext * mu_0 * (1 + c/100)
I2 = JA_ext * mu_0 * (1 - c/100)

#Initial activity (firing rate) of each population
S1 = 1.1
S2 = 1.1
current_noise = np.zeros(2)

#Noisy sources from each ionic channel 
def noisy_channel(noises,tau_AMPA = 2, s_noise=0.05):
    #a two-dimensional array of noisy displacements for the current run
    eta =  np.random.rand(2,1) * np.sqrt(tau_AMPA * s_noise ** 2)
    #Inoise1' = -Inoise1 + eta1 * sqrt(tau_AMPA * s_noise^2)
    #Inoise2' = -Inoise2 + eta2 * sqrt(tau_AMPA * s_noise^2)
    return sum(-noises,eta) #[ionic_noises]

#Summing contributions to the hidden state (x) of each population
#x1 = JN11 * S1 - JN12 * S2 + I0 + I1 + Inoise1
#x2 = JN22 * S2 - JN21 * S1 + I0 + I2 + Inoise2

def build_hidden_states(couplings,population_activities,inputs,ionic_noises,external_sources = I0):
    #returns a 2x1 array, each element being the hidden state activity (firing rate) of one excitatory population
    return couplings @ population_activities + external_sources + inputs + ionic_noises #[hidden_states]

#yet to port/solve

#these two will become the function below
#S1' = -S1/tau_S + (1 - S1) * g * H(x1)/1000
#S2' = -S2/tau_S + (1 - S2) * g * H(x2)/1000

def population_states(hidden_states,population_activities,g,tau_S = 100):
    return -population_activities/tau_S + (np.ones(2) - population_activities) * g * input_output_relation(hidden_states)/1000

In [105]:
#xpp former parameters
#@ dt=0.1
#@ xp=S1,yp=S2,xlo=-0.1,ylo=-0.1,xhi=1.1,yhi=1.1,total=10000